In [1]:
import pandas as pd
import datetime as dt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
events = pd.read_csv('fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv')

/home/euge/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_data = pd.read_csv('fiuba-trocafone-tp2-final-set/labels_training_set.csv')

In [4]:
test_kaggle = pd.read_csv('fiuba-trocafone-tp2-final-set/trocafone_kaggle_test.csv')

FileNotFoundError: File b'fiuba-trocafone-tp2-final-set/trocafone_kaggle_test.csv' does not exist

In [6]:
events['week'] = pd.to_datetime(events['timestamp']).dt.week

In [7]:
events['week'].value_counts()

21    540477
20    495049
22    408822
19    166141
18    118139
17     96538
16     67727
15     65513
14     60947
13     55405
12     55377
11     40157
10     31867
9      28420
8      21058
5      16097
7      15546
6      15323
3      12688
2      12122
4      11537
1       6731
Name: week, dtype: int64

In [8]:
#SEPARO EN PERIODOS DE DOS SEMANAS
def periodo (week):
    return (week+1)//2

events['periodo'] = events['week'].apply(lambda week: periodo(week))

In [9]:
events['periodo'].value_counts()

11    949299
10    661190
9     214677
8     133240
7     116352
6      95534
5      60287
4      36604
3      31420
2      24225
1      18853
Name: periodo, dtype: int64

# Feature: Cantidad de eventos por usuario 

Solo trabajo con alguno de los periodos

In [9]:
eventos = events.loc[events['periodo']>6]

In [10]:
eventos= eventos[['person','periodo','timestamp']]

In [11]:
eventos_por_periodo = eventos.groupby(['periodo','person']).count().unstack(level=0)

In [12]:
eventos_por_periodo.reset_index(inplace=True)

In [13]:
eventos_por_periodo.columns = [col[1] for col in eventos_por_periodo.columns]

In [14]:
eventos_por_periodo.rename(columns={'':'person'},inplace=True)
eventos_por_periodo.fillna(value=0,inplace=True)

In [15]:
eventos_por_periodo.rename(columns={7:'eventos_periodo_7',8:'eventos_periodo_8',9:'eventos_periodo_9',10:'eventos_periodo_10',11:'eventos_periodo_11'})

,person,eventos_periodo_7,eventos_periodo_8,eventos_periodo_9,eventos_periodo_10,eventos_periodo_11
0,0008ed71,0.0,0.0,0.0,6.0,0.0
1,00091926,0.0,0.0,33.0,299.0,116.0
2,00091a7a,10.0,0.0,0.0,0.0,0.0
3,000ba417,0.0,0.0,0.0,68.0,138.0
4,000c79fe,0.0,0.0,0.0,0.0,17.0
5,000e4d9e,0.0,0.0,0.0,195.0,216.0
6,000e619d,0.0,0.0,0.0,68.0,0.0
7,001001be,0.0,0.0,0.0,67.0,1.0
8,0010e89a,0.0,0.0,0.0,0.0,4.0
9,0016c4b5,0.0,0.0,0.0,0.0,4.0


In [16]:
train_data = pd.merge(train_data,eventos_por_periodo,on='person',how='left')
test_kaggle = pd.merge(test_kaggle,eventos_por_periodo,on='person',how='left')

NameError: name 'test_kaggle' is not defined

In [17]:
test_kaggle.fillna(value=0,inplace=True)

NameError: name 'test_kaggle' is not defined

# Gradient boosting

In [18]:
gBoosting = GradientBoostingClassifier(n_estimators=20, max_features=4,max_depth=10)

In [19]:
X = train_data[['eventos_periodo_7', 'eventos_periodo_8','eventos_periodo_9', 'eventos_periodo_10', 'eventos_periodo_11']]
Y = train_data['label']

X_train, X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)

KeyError: "['eventos_periodo_7' 'eventos_periodo_8' 'eventos_periodo_9'\n 'eventos_periodo_10' 'eventos_periodo_11'] not in index"

In [20]:
gBoosting.fit(X_train,Y_train)

NameError: name 'X_train' is not defined

In [21]:
gBoosting.score(X_test,Y_test)

NameError: name 'X_test' is not defined

In [ ]:
test_kaggle['prediction'] = gBoosting.predict(test_kaggle[['eventos_periodo_7', 'eventos_periodo_8','eventos_periodo_9', 'eventos_periodo_10', 'eventos_periodo_11']])

In [ ]:
test

In [ ]:
submit_kaggle = pd.DataFrame({'person':test_kaggle['person'],'label':test_kaggle['prediction']})
submit_kaggle.reset_index(inplace = True)
submit_kaggle.drop(columns='index',inplace=True)
submit_kaggle.set_index(keys='person',inplace=True)

In [ ]:
submit_kaggle.to_csv('submit_kaggle.csv')

# Feature: region - resolucion de pantalla

In [35]:
region = events[['person','region','screen_resolution']].dropna()
region

,person,region,screen_resolution
2136629,4640420b,Parana,320x570
2136630,4640420b,Parana,320x570
2136631,4640420b,Unknown,1280x1024
2136632,4640420b,Parana,800x1280
2136633,4640420b,Parana,800x1280
2136634,4640420b,Parana,320x570
2136635,4640420b,Parana,800x1280
2136636,4640420b,Sao Paulo,320x570
2136637,4640420b,Unknown,1280x1024
2136638,4640420b,Parana,800x1280


In [36]:
region['region_resolucion'] = region['region']+region['screen_resolution']
region_resolucion = region[['person','region_resolucion']]
region_resolucion

,person,region_resolucion
2136629,4640420b,Parana320x570
2136630,4640420b,Parana320x570
2136631,4640420b,Unknown1280x1024
2136632,4640420b,Parana800x1280
2136633,4640420b,Parana800x1280
2136634,4640420b,Parana320x570
2136635,4640420b,Parana800x1280
2136636,4640420b,Sao Paulo320x570
2136637,4640420b,Unknown1280x1024
2136638,4640420b,Parana800x1280


In [37]:
region_resolucion = region_resolucion.drop_duplicates()
region_resolucion

,person,region_resolucion
2136629,4640420b,Parana320x570
2136631,4640420b,Unknown1280x1024
2136632,4640420b,Parana800x1280
2136636,4640420b,Sao Paulo320x570
2136640,4640420b,Parana320x522
2136652,4640420b,Santa Catarina1280x1024
2136655,4640420b,Santa Catarina320x522
2136659,4640420b,Sao Paulo1280x1024
2136660,f35dddc8,Piaui360x640
2136661,c82ecb12,Rio de Janeiro360x640


In [38]:
region_resolucion['region_resolucion_h'] = region_resolucion['region_resolucion'].apply(lambda a: hash(a))
region_resolucion

/home/euge/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,person,region_resolucion,region_resolucion_h
2136629,4640420b,Parana320x570,-3486592093772705005
2136631,4640420b,Unknown1280x1024,-5735015092186701903
2136632,4640420b,Parana800x1280,-4233600664636165113
2136636,4640420b,Sao Paulo320x570,-8836238896939786766
2136640,4640420b,Parana320x522,7068915030938652175
2136652,4640420b,Santa Catarina1280x1024,1630208838237318951
2136655,4640420b,Santa Catarina320x522,-4893498236986548312
2136659,4640420b,Sao Paulo1280x1024,-7074889735127548929
2136660,f35dddc8,Piaui360x640,4736145904860099221
2136661,c82ecb12,Rio de Janeiro360x640,1844095417007047287


In [39]:
train_data = train_data.merge(region_resolucion,on='person',how='left')

In [ ]:
train_data.to_csv('features/train_data.csv')

In [ ]:
test_kaggle = test_kaggle.merge(region_resolucion,on='person',how='left')

In [ ]:
test_kaggle.to_csv('test_kaggle.csv')

# Feature : Condicion del producto

In [64]:
condicion_general = events[['person','event','condition','periodo']]

In [65]:
condicion_general = condicion_general.groupby(['periodo','person','condition']).count().unstack(level=0)
condicion_general

event                                              \
periodo                        1     2     3     4     5      6      7     8    
person   condition                                                              
0008ed71 Bom                  NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Muito Bom            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
00091926 Bom                  NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Excelente            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Muito Bom            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Novo                 NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
00091a7a Bom                  NaN   NaN   NaN   NaN   NaN    NaN    1.0   NaN   
         Excelente            NaN   NaN   NaN   NaN   NaN    NaN    2.0   NaN   
000ba417 Bom                  NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Excelente            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Muito Bom            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
000c79fe Bom                  NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
000e4d9e Bom                  NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Excelente            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Muito Bom            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
000e619d Bom                  NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Excelente            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Muito Bom            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
001001be Bom                  NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Bom - Sem Touch ID   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Excelente            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Muito Bom            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
0010e89a Bom                  NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
0016c4b5 Bom                  NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
001802e4 Bom                  NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Excelente            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
001804a2 Bom                  NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Muito Bom            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
0019e639 Bom                  NaN   6.0  27.0   4.0   NaN    NaN    NaN   NaN   
         Excelente            NaN   NaN   1.0   NaN   NaN    NaN    NaN   NaN   
...                           ...   ...   ...   ...   ...    ...    ...   ...   
ffed0342 Bom                  NaN   NaN   NaN   NaN  14.0    NaN    NaN   NaN   
         Bom - Sem Touch ID   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Excelente            NaN   NaN   NaN   NaN  27.0    NaN    NaN   NaN   
         Muito Bom            NaN   NaN   NaN   NaN  23.0    NaN    NaN   NaN   
ffed3d0e Bom                  NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
ffee0f18 Bom                 25.0  22.0  52.0  57.0  57.0  104.0   47.0  16.0   
         Excelente           39.0  68.0  89.0  70.0  89.0  149.0  125.0  34.0   
         Muito Bom           13.0  12.0  48.0  36.0  59.0   53.0   30.0  10.0   
         Novo                 NaN   NaN   NaN   NaN   NaN    NaN    NaN   2.0   
ffef83e6 Bom                  NaN   NaN   NaN   NaN   NaN    NaN    5.0   5.0   
         Excelente            NaN   NaN   NaN   NaN   NaN    NaN    8.0  20.0   
         Muito Bom            NaN   NaN   NaN   NaN   NaN    NaN    4.0   8.0   
fff1659c Bom                  NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Excelente            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
         Muito Bom            NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   
fff1b11a Bom                  NaN   NaN   NaN   NaN   NaN  

In [66]:
condicion_general = condicion_general.unstack()
condicion_general.fillna(value=0,inplace=True)
condicion_general = condicion_general['event']

In [67]:
condicion_general

periodo      1                                                 2   \
condition   Bom Bom - Sem Touch ID Excelente Muito Bom Novo   Bom   
person                                                              
0008ed71    0.0                0.0       0.0       0.0  0.0   0.0   
00091926    0.0                0.0       0.0       0.0  0.0   0.0   
00091a7a    0.0                0.0       0.0       0.0  0.0   0.0   
000ba417    0.0                0.0       0.0       0.0  0.0   0.0   
000c79fe    0.0                0.0       0.0       0.0  0.0   0.0   
000e4d9e    0.0                0.0       0.0       0.0  0.0   0.0   
000e619d    0.0                0.0       0.0       0.0  0.0   0.0   
001001be    0.0                0.0       0.0       0.0  0.0   0.0   
0010e89a    0.0                0.0       0.0       0.0  0.0   0.0   
0016c4b5    0.0                0.0       0.0       0.0  0.0   0.0   
001802e4    0.0                0.0       0.0       0.0  0.0   0.0   
001804a2    0.0                0.0       0.0       0.0  0.0   0.0   
0019e639    0.0                0.0       0.0       0.0  0.0   6.0   
001a2273    0.0                0.0       0.0       0.0  0.0   0.0   
001b0bf9    0.0                0.0       0.0       0.0  0.0   0.0   
001ca5ee    0.0                0.0       0.0       0.0  0.0   0.0   
001dfc31    0.0                0.0       0.0       0.0  0.0   0.0   
001e9aea    0.0                0.0       0.0       0.0  0.0   0.0   
0020152e    0.0                0.0       0.0       0.0  0.0   0.0   
00202cdb    0.0                0.0       0.0       0.0  0.0   0.0   
0022965d    0.0                0.0       0.0       0.0  0.0   0.0   
0027574e    0.0                0.0       0.0       0.0  0.0   0.0   
00295d74    0.0                0.0       0.0       0.0  0.0   0.0   
002aea56    0.0                0.0       0.0       0.0  0.0  22.0   
002b0188    0.0                0.0       0.0       0.0  0.0   0.0   
002e74b2    0.0                0.0       0.0       0.0  0.0   0.0   
002ed810    0.0                0.0       0.0       0.0  0.0   0.0   
002fb981    0.0                0.0       0.0       0.0  0.0   0.0   
00317e49    0.0                0.0       0.0       0.0  0.0   0.0   
0031c75f    0.0                0.0       0.0       0.0  0.0   0.0   
...         ...                ...       ...       ...  ...   ...   
ffc670b3    0.0                0.0       0.0       0.0  0.0   0.0   
ffcaae19    0.0                0.0       0.0       0.0  0.0   0.0   
ffcaeaae    0.0                0.0       0.0       0.0  0.0   0.0   
ffccd5a1    0.0                0.0       0.0       0.0  0.0   0.0   
ffcf952b    0.0                0.0       0.0       0.0  0.0   0.0   
ffd41214    0.0                0.0       0.0       0.0  0.0   0.0   
ffd62616    0.0                0.0       0.0       0.0  0.0   0.0   
ffd9756a    0.0                0.0       0.0       0.0  0.0   0.0   
ffda14ca    0.0                0.0       0.0       0.0  0.0   0.0   
ffdafc72    0.0                0.0       0.0       0.0  0.0   0.0   
ffddd0ec    0.0                0.0       0.0       0.0  0.0   0.0   
ffde3628    0.0                0.0       0.0       0.0  0.0   0.0   
ffe066cc    0.0                0.0       0.0       0.0  0.0   0.0   
ffe53446    0.0                0.0       0.0       0.0  0.0   0.0   
ffe7e848    0.0                0.0       0.0       0.0  0.0   0.0   
ffeb80c5    0.0                0.0       0.0       0.0  0.0   0.0   
ffebdbc9    0.0                0.0       0.0       0.0  0.0   0.0   
ffecdf29    0.0                0.0       0.0       0.0  0.0   0.0   
ffecea3d    0.0                0.0       0.0       0.0  0.0   0.0   
ffed0342    0.0                0.0       0.0       0.0  0.0   0.0   
ffed3d0e    0.0                0.0       0.0       0.0  0.0   0.0   
ffee0f18   25.0                0.0      39.0      13.0  0.0  22.0   
ffef83e6    0.0                0.0       0.0       0.0  0.0   0.0   
fff1659c    0.0                0.0       0.0       0.0  0.0   0.0  

In [68]:
condicion_general.reset_index(inplace=True)
condicion_general

periodo      person     1                                                 2  \
condition             Bom Bom - Sem Touch ID Excelente Muito Bom Novo   Bom   
0          0008ed71   0.0                0.0       0.0       0.0  0.0   0.0   
1          00091926   0.0                0.0       0.0       0.0  0.0   0.0   
2          00091a7a   0.0                0.0       0.0       0.0  0.0   0.0   
3          000ba417   0.0                0.0       0.0       0.0  0.0   0.0   
4          000c79fe   0.0                0.0       0.0       0.0  0.0   0.0   
5          000e4d9e   0.0                0.0       0.0       0.0  0.0   0.0   
6          000e619d   0.0                0.0       0.0       0.0  0.0   0.0   
7          001001be   0.0                0.0       0.0       0.0  0.0   0.0   
8          0010e89a   0.0                0.0       0.0       0.0  0.0   0.0   
9          0016c4b5   0.0                0.0       0.0       0.0  0.0   0.0   
10         001802e4   0.0                0.0       0.0       0.0  0.0   0.0   
11         001804a2   0.0                0.0       0.0       0.0  0.0   0.0   
12         0019e639   0.0                0.0       0.0       0.0  0.0   6.0   
13         001a2273   0.0                0.0       0.0       0.0  0.0   0.0   
14         001b0bf9   0.0                0.0       0.0       0.0  0.0   0.0   
15         001ca5ee   0.0                0.0       0.0       0.0  0.0   0.0   
16         001dfc31   0.0                0.0       0.0       0.0  0.0   0.0   
17         001e9aea   0.0                0.0       0.0       0.0  0.0   0.0   
18         0020152e   0.0                0.0       0.0       0.0  0.0   0.0   
19         00202cdb   0.0                0.0       0.0       0.0  0.0   0.0   
20         0022965d   0.0                0.0       0.0       0.0  0.0   0.0   
21         0027574e   0.0                0.0       0.0       0.0  0.0   0.0   
22         00295d74   0.0                0.0       0.0       0.0  0.0   0.0   
23         002aea56   0.0                0.0       0.0       0.0  0.0  22.0   
24         002b0188   0.0                0.0       0.0       0.0  0.0   0.0   
25         002e74b2   0.0                0.0       0.0       0.0  0.0   0.0   
26         002ed810   0.0                0.0       0.0       0.0  0.0   0.0   
27         002fb981   0.0                0.0       0.0       0.0  0.0   0.0   
28         00317e49   0.0                0.0       0.0       0.0  0.0   0.0   
29         0031c75f   0.0                0.0       0.0       0.0  0.0   0.0   
...             ...   ...                ...       ...       ...  ...   ...   
38220      ffc670b3   0.0                0.0       0.0       0.0  0.0   0.0   
38221      ffcaae19   0.0                0.0       0.0       0.0  0.0   0.0   
38222      ffcaeaae   0.0                0.0       0.0       0.0  0.0   0.0   
38223      ffccd5a1   0.0                0.0       0.0       0.0  0.0   0.0   
38224      ffcf952b   0.0                0.0       0.0       0.0  0.0   0.0   
38225      ffd41214   0.0                0.0       0.0       0.0  0.0   0.0   
38226      ffd62616   0.0                0.0       0.0       0.0  0.0   0.0   
38227      ffd9756a   0.0                0.0       0.0       0.0  0.0   0.0   
38228      ffda14ca   0.0                0.0       0.0       0.0  0.0   0.0   
38229      ffdafc72   0.0                0.0       0.0       0.0  0.0   0.0   
38230      ffddd0ec   0.0                0.0       0.0       0.0  0.0   0.0   
38231      ffde3628   0.0                0.0       0.0       0.0  0.0   0.0   
38232      ffe066cc   0.0                0.0       0.0       0.0  0.0   0.0   
38233      ffe53446   0.0                0.0       0.0       0.0  0.0   0.0   
38234      ffe7e848   0.0                0.0       0.0       0.0  0.0   0.0   
38235      ffeb80c5   0.0                0.0       0.0       0.0  0.0   0.0   
38236      ffebdbc9   0.0                0.0       0.0       0.0  0.0   0.0   
38237      ffecdf29   0.0                0.0      

In [69]:
train_data = train_data.merge(condicion_general,on='person',how='left')
train_data

/home/euge/anaconda/lib/python3.6/site-packages/pandas/core/reshape/merge.py:551: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/home/euge/anaconda/lib/python3.6/site-packages/pandas/core/reshape/merge.py:862: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  self.right = self.right.drop(right_drop, axis=1)


,person,label,region_resolucion,region_resolucion_h,"(1, Bom)","(1, Bom - Sem Touch ID)","(1, Excelente)","(1, Muito Bom)","(1, Novo)","(2, Bom)",...,"(10, Bom)","(10, Bom - Sem Touch ID)","(10, Excelente)","(10, Muito Bom)","(10, Novo)","(11, Bom)","(11, Bom - Sem Touch ID)","(11, Excelente)","(11, Muito Bom)","(11, Novo)"
0,0566e9c1,0,Rio de Janeiro320x568,-1.166954e+18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,2.0,1.0,6.0,0.0
1,0566e9c1,0,Parana320x568,-2.881457e+18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,2.0,1.0,6.0,0.0
2,6ec7ee77,0,Unknown360x640,3.632630e+18,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abe7a2fb,0,Rio de Janeiro360x640,1.844095e+18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,2.0,2.0,0.0
4,abe7a2fb,0,Sao Paulo360x640,3.022559e+18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,2.0,2.0,0.0
5,34728364,0,Sao Paulo360x640,3.022559e+18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,13.0,0.0,7.0,4.0,0.0
6,87ed62de,0,Santa Catarina1152x864,-6.620510e+18,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,db2c4d27,1,Rio de Janeiro360x640,1.844095e+18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,db2c4d27,1,Unknown1366x768,2.370245e+18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,db2c4d27,1,Unknown360x640,3.632630e+18,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Feature: Promedio de ingresos al sitio 

In [73]:
visitas = events.loc[events['event']=='visited site']

In [74]:
visitas = visitas[['timestamp','person','event','periodo']]

In [75]:
visitas['timestamp'] = pd.to_datetime(visitas['timestamp'])
visitas['dia'] = visitas['timestamp'].dt.dayofweek

In [81]:
visitas_por_dia = visitas.groupby(['periodo','person','dia'])['event'].count().unstack(level=0)
visitas_por_dia

periodo        1    2    3    4    5    6    7    8    9    10   11
person   dia                                                       
0008ed71 3    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0  NaN
00091926 0    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0
         1    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0  2.0
         2    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0  1.0
         3    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  3.0  2.0
         4    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  3.0  NaN
         5    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0  1.0  2.0
         6    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  8.0  2.0
00091a7a 0    NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN  NaN  NaN  NaN
000ba417 3    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  3.0  1.0
         5    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0
000c79fe 1    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
000e4d9e 0    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0
         1    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  3.0
         2    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  3.0  3.0
         3    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
         4    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
000e619d 1    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN
         2    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN
         3    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN
         4    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0  NaN
001001be 1    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
         2    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN
         5    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN
0010e89a 2    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
0016c4b5 1    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
001802e4 3    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
001804a2 0    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
0019e639 0    NaN  NaN  NaN  1.0  NaN  NaN  NaN  NaN  1.0  1.0  NaN
         1    NaN  NaN  NaN  2.0  NaN  NaN  NaN  NaN  NaN  2.0  2.0
...           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
ffee0f18 3    NaN  3.0  1.0  3.0  3.0  5.0  3.0  3.0  2.0  1.0  1.0
         4    NaN  2.0  2.0  5.0  8.0  3.0  1.0  5.0  1.0  NaN  NaN
         5    1.0  2.0  7.0  2.0  3.0  3.0  1.0  2.0  1.0  NaN  NaN
         6    3.0  3.0  3.0  2.0  4.0  1.0  NaN  4.0  NaN  NaN  NaN
ffef83e6 0    NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  1.0  NaN  1.0
         1    NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  NaN  2.0  NaN
         2    NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  1.0  NaN  NaN
         3    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  NaN
         4    NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0  NaN  1.0
         5    NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0  1.0  1.0  NaN
         6    NaN  NaN  NaN  NaN  NaN  NaN  1.0  2.0  NaN  NaN  1.0
fff1659c 0    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
         1    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
         2    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.0
         3    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
         6    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
fff1b11a 0    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
fff1caee 1    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0
fff2bdde 2    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0  NaN
         3    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN
         6    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN  NaN
fff54d61 4    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN
fff72025 0    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  3.0  1.0
         1    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  NaN
         2    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  2.0  NaN
         4    NaN  NaN  NaN  NaN  NaN  NaN  NaN  1.0  1.

In [82]:
visitas_por_dia.reset_index(inplace=True)
visitas_por_dia = visitas_por_dia.rename(columns={1:'visitas_por_dia_1',
                                                 2:'visitas_por_dia_2',
                                                 3:'visitas_por_dia_3',
                                                 4:'visitas_por_dia_4',
                                                 5:'visitas_por_dia_5',
                                                 6:'visitas_por_dia_6',
                                                 7:'visitas_por_dia_7',
                                                 8:'visitas_por_dia_8',
                                                 9:'visitas_por_dia_9',
                                                 10:'visitas_por_dia_10',
                                                 11:'visitas_por_dia_11',
                                                 })
visitas_por_dia = visitas_por_dia.fillna(0)
visitas_por_dia

periodo,person,dia,visitas_por_dia_1,visitas_por_dia_2,visitas_por_dia_3,visitas_por_dia_4,visitas_por_dia_5,visitas_por_dia_6,visitas_por_dia_7,visitas_por_dia_8,visitas_por_dia_9,visitas_por_dia_10,visitas_por_dia_11
0,0008ed71,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,00091926,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,00091926,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
3,00091926,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0
4,00091926,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,2.0
5,00091926,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
6,00091926,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0
7,00091926,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,2.0
8,00091a7a,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,000ba417,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0


In [84]:
promedio_visitas_por_dia = visitas_por_dia.groupby('person').agg({'visitas_por_dia_1':'mean',
                                                 'visitas_por_dia_2':'mean',
                                                 'visitas_por_dia_3':'mean',
                                                 'visitas_por_dia_4':'mean',
                                                 'visitas_por_dia_5':'mean',
                                                 'visitas_por_dia_6':'mean',
                                                 'visitas_por_dia_7':'mean',
                                                 'visitas_por_dia_8':'mean',
                                                 'visitas_por_dia_9':'mean',
                                                 'visitas_por_dia_10':'mean',
                                                 'visitas_por_dia_11':'mean',})

In [85]:
promedio_visitas_por_dia

,visitas_por_dia_1,visitas_por_dia_2,visitas_por_dia_3,visitas_por_dia_4,visitas_por_dia_5,visitas_por_dia_6,visitas_por_dia_7,visitas_por_dia_8,visitas_por_dia_9,visitas_por_dia_10,visitas_por_dia_11
person,,,,,,,,,,,
0008ed71,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000
00091926,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.571429,2.857143,1.428571
00091a7a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
000ba417,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000,1.500000
000c79fe,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
000e4d9e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.600000,2.000000
000e619d,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.250000,0.000000
001001be,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.333333
0010e89a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [86]:
promedio_visitas_por_dia.reset_index(inplace=True)

In [88]:
promedio_visitas_por_dia

,person,visitas_por_dia_1,visitas_por_dia_2,visitas_por_dia_3,visitas_por_dia_4,visitas_por_dia_5,visitas_por_dia_6,visitas_por_dia_7,visitas_por_dia_8,visitas_por_dia_9,visitas_por_dia_10,visitas_por_dia_11
0,0008ed71,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000
1,00091926,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.571429,2.857143,1.428571
2,00091a7a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
3,000ba417,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.500000,1.500000
4,000c79fe,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
5,000e4d9e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.600000,2.000000
6,000e619d,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.250000,0.000000
7,001001be,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.333333
8,0010e89a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
9,0016c4b5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


# Feature: Cantidad promedio de compras por usuario por periodo

In [89]:
conversiones = events[['timestamp','person','event','periodo']]
conversiones = conversiones.loc[conversiones['event']=='conversion']
conversiones['timestamp'] = pd.to_datetime(conversiones['timestamp'])
conversiones['mes'] = conversiones['timestamp'].dt.month

In [90]:
conversiones

,timestamp,person,event,periodo,mes
195,2018-05-18 01:12:20,49c19e32,conversion,10,5
234,2018-05-18 01:05:23,49c19e32,conversion,10,5
724,2018-05-29 07:51:14,39df97e0,conversion,11,5
744,2018-05-29 07:13:30,380c0e60,conversion,11,5
1198,2018-05-29 19:31:46,35ee0cc8,conversion,11,5
2124,2018-05-29 18:36:30,c5a0bc36,conversion,11,5
2550,2018-05-29 17:26:11,6f0826b2,conversion,11,5
3263,2018-05-29 17:20:21,576d1af6,conversion,11,5
3391,2018-05-29 19:02:08,04b13fd2,conversion,11,5
3558,2018-05-29 18:46:21,bf15584a,conversion,11,5


In [122]:
conversiones_por_periodo = conversiones.groupby(['periodo','person'])['event'].count().to_frame()
conversiones_por_periodo.reset_index(inplace=True)
conversiones_por_periodo.fillna(0)

,periodo,person,event
0,1,027870af,1
1,1,064ec6ea,1
2,1,08343c0e,1
3,1,0a5879fc,1
4,1,11e3fcc3,2
5,1,123fde23,1
6,1,29e1938c,1
7,1,343618be,1
8,1,3575e80c,2
9,1,40f9b318,1


In [123]:
conversiones_por_periodo['mean_conversion'] = conversiones_por_periodo['event']/11
conversiones_por_periodo

,periodo,person,event,mean_conversion
0,1,027870af,1,0.090909
1,1,064ec6ea,1,0.090909
2,1,08343c0e,1,0.090909
3,1,0a5879fc,1,0.090909
4,1,11e3fcc3,2,0.181818
5,1,123fde23,1,0.090909
6,1,29e1938c,1,0.090909
7,1,343618be,1,0.090909
8,1,3575e80c,2,0.181818
9,1,40f9b318,1,0.090909


In [124]:
conversiones_por_periodo

,periodo,person,event,mean_conversion
0,1,027870af,1,0.090909
1,1,064ec6ea,1,0.090909
2,1,08343c0e,1,0.090909
3,1,0a5879fc,1,0.090909
4,1,11e3fcc3,2,0.181818
5,1,123fde23,1,0.090909
6,1,29e1938c,1,0.090909
7,1,343618be,1,0.090909
8,1,3575e80c,2,0.181818
9,1,40f9b318,1,0.090909


# Feature : Personas que vieron el mismo producto mas de 5 veces en el mismo periodo

In [137]:
viewed_product = events.loc[events['event']== 'viewed product']
cantidad_de_visitas_a_producto = viewed_product.groupby(['periodo','person','model'])['event'].count().unstack(level=0)
cantidad_de_visitas_a_producto.reset_index(inplace=True)

In [138]:
cantidad_de_visitas_a_producto.rename(columns={1:'mean_viewed_product_1',
                                              2:'mean_viewed_product_2',
                                              3:'mean_viewed_product_3',
                                              4:'mean_viewed_product_4',
                                              5:'mean_viewed_product_5',
                                              6:'mean_viewed_product_6',
                                              7:'mean_viewed_product_7',
                                              8:'mean_viewed_product_8',
                                              9:'mean_viewed_product_9',
                                              10:'mean_viewed_product_10',
                                              11:'mean_viewed_product_11',
                                              },inplace=True)

cantidad_de_visitas_a_producto = cantidad_de_visitas_a_producto.fillna(0)

In [139]:
cantidad_de_visitas_a_producto

periodo,person,model,mean_viewed_product_1,mean_viewed_product_2,mean_viewed_product_3,mean_viewed_product_4,mean_viewed_product_5,mean_viewed_product_6,mean_viewed_product_7,mean_viewed_product_8,mean_viewed_product_9,mean_viewed_product_10,mean_viewed_product_11
0,00091926,LG G4 H815P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,00091926,LG G5 SE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,00091926,Lenovo Vibe K5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,00091926,Motorola Moto G4 Play DTV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
4,00091926,Motorola Moto G5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
5,00091926,Motorola Moto X Force,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,00091926,Motorola Moto X Play 4G Dual,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
7,00091926,Motorola Moto X Style,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,24.0,6.0
8,00091926,Motorola Moto X2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0
9,00091926,Samsung Galaxy A5 2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [143]:
feature_visitas_a_producto_promedio = cantidad_de_visitas_a_producto.groupby('person').mean()

In [144]:
feature_visitas_a_producto_promedio

periodo,mean_viewed_product_1,mean_viewed_product_2,mean_viewed_product_3,mean_viewed_product_4,mean_viewed_product_5,mean_viewed_product_6,mean_viewed_product_7,mean_viewed_product_8,mean_viewed_product_9,mean_viewed_product_10,mean_viewed_product_11
person,,,,,,,,,,,
00091926,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.805556,6.944444,2.583333
00091a7a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
000ba417,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.461538,4.423077
000c79fe,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
000e4d9e,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.135135,5.027027
000e619d,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.217391,0.000000
001001be,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.666667,0.000000
0010e89a,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
0016c4b5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


In [145]:
feature_visitas_a_producto_promedio.reset_index(inplace=True)

# Feature : Colores de los dispositivos por periodo

In [73]:
color = events[['person','color','event','periodo']]

In [74]:
color = color.loc[color['event']=='viewed product']

In [75]:
color = color.groupby(['periodo','person','color'])['event'].count().unstack(level=0)

In [76]:
color.reset_index(inplace=True)

In [77]:
color = color.loc[(color['color'] == 'Dourado') | (color['color'] == 'Preto') | (color['color']== 'Branco')]
color = color.fillna(0)

In [78]:
for x in range(1,12):
    color[x] = color[x].apply(lambda a: 1 if a != 0 else a)
color

periodo,person,color,1,2,3,4,5,6,7,8,9,10,11
5,00091926,Branco,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,00091926,Dourado,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
13,00091926,Preto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
21,00091a7a,Dourado,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25,000ba417,Branco,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
32,000ba417,Dourado,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
34,000ba417,Preto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
46,000e4d9e,Branco,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
48,000e4d9e,Dourado,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
53,000e4d9e,Preto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [79]:
color.drop_duplicates(subset='person', inplace=True)

In [80]:
color.rename(columns={1:'vio_color_mas_vendido_1',
                                              2:'vio_color_mas_vendido_2',
                                              3:'vio_color_mas_vendido_3',
                                              4:'vio_color_mas_vendido_4',
                                              5:'vio_color_mas_vendido_5',
                                              6:'vio_color_mas_vendido_6',
                                              7:'vio_color_mas_vendido_7',
                                              8:'vio_color_mas_vendido_8',
                                              9:'vio_color_mas_vendido_9',
                                              10:'vio_color_mas_vendido_10',
                                              11:'vio_color_mas_vendido_11',
                                              },inplace=True)

color = color.fillna(0)
color

periodo,person,color,mean_viewed_product_1,vio_color_mas_vendido_2,vio_color_mas_vendido_3,vio_color_mas_vendido_4,vio_color_mas_vendido_5,vio_color_mas_vendido_6,vio_color_mas_vendido_7,vio_color_mas_vendido_8,vio_color_mas_vendido_9,vio_color_mas_vendido_10,vio_color_mas_vendido_11
5,00091926,Branco,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
21,00091a7a,Dourado,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25,000ba417,Branco,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
46,000e4d9e,Branco,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
57,000e619d,Branco,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
66,001001be,Dourado,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
68,0010e89a,Preto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
70,001802e4,Dourado,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
74,0019e639,Branco,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
87,001a2273,Preto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Feature: Cantidad de productos de la misma marca vistos por usuario

In [81]:
brand_listing = events[['person','event','periodo']]

In [82]:
brand_listing = brand_listing.loc[brand_listing['event']=='brand listing']

In [83]:
brand_listing = brand_listing.groupby(['periodo', 'person'])['event'].count().unstack(level=0)
brand_listing.reset_index(inplace=True)
brand_listing = brand_listing.fillna(0)
brand_listing

periodo,person,1,2,3,4,5,6,7,8,9,10,11
0,00091926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,3.0
1,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2,000ba417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,9.0
3,000e4d9e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,14.0
4,000e619d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0
5,0019e639,0.0,17.0,17.0,0.0,0.0,0.0,0.0,0.0,33.0,66.0,32.0
6,001b0bf9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,001ca5ee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,1.0
8,001e9aea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
9,0027574e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0


In [84]:
for x in range(1,12):
    brand_listing[x] = brand_listing[x].apply(lambda a: 1 if a != 0 else a)
brand_listing

periodo,person,1,2,3,4,5,6,7,8,9,10,11
0,00091926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,000ba417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,000e4d9e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,000e619d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0019e639,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
6,001b0bf9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,001ca5ee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
8,001e9aea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,0027574e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [85]:
brand_listing.drop_duplicates(subset = 'person', inplace=True)

brand_listing.rename(columns={1:'visito_brand_listing_1',
                                              2:'visito_brand_listing_2',
                                              3:'visito_brand_listing_3',
                                              4:'visito_brand_listing_4',
                                              5:'visito_brand_listing_5',
                                              6:'visito_brand_listing_6',
                                              7:'visito_brand_listing_7',
                                              8:'visito_brand_listing_8',
                                              9:'visito_brand_listing_9',
                                              10:'visito_brand_listing_10',
                                              11:'visito_brand_listing_11',
                                              },inplace=True)

brand_listing = brand_listing.fillna(0)
brand_listing

periodo,person,visito_brand_listing_1,visito_brand_listing_2,visito_brand_listing_3,visito_brand_listing_4,visito_brand_listing_5,visito_brand_listing_6,visito_brand_listing_7,visito_brand_listing_8,visito_brand_listing_9,visito_brand_listing_10,visito_brand_listing_11
0,00091926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,000ba417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,000e4d9e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,000e619d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0019e639,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
6,001b0bf9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,001ca5ee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
8,001e9aea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,0027574e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
